# Download Dataset

In [ ]:
!gdown --id 1zl2nsDqjSjHCwVnl2d_H3BPd0_HikW3P --output hw2_data.zip
!unzip -q hw2_data.zip
!rm hw2_data.zip

Downloading...
From: https://drive.google.com/uc?id=1zl2nsDqjSjHCwVnl2d_H3BPd0_HikW3P
To: /content/hw2_data.zip
100% 642M/642M [00:03<00:00, 179MB/s]


# Bash test


In [ ]:
!mkdir './test_repo'

In [ ]:
!bash hw2_p2.sh './test_repo'

In [ ]:
# %cd './output_repo'
# %rm *.png
%cd ..

/content


In [ ]:
!python hw2_2.py

# Import Packages

In [1]:
import glob
import numpy as np
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from PIL import Image
from torch.utils import data
from torch.utils.data import DataLoader
from torchvision.transforms import transforms

# Define Dataset Class

In [ ]:
# dataset class
class mnistm_dataset(data.Dataset):
    def __init__(self, inputs_path: list, labels: list, transforms):
        self.inputs_path = inputs_path
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.inputs_path)
    
    def __getitem__(self, index: int):
        
        input_path = self.inputs_path[index]
        input = Image.open(input_path)
        label = self.labels[index]

        return self.transforms(input), label

NameError: ignored

# Get Dataset

In [ ]:
def get_dataset(batch_size, n_workers):

    train_transforms = transforms.Compose([
        # transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ])
    
    path = './hw2_data/digits/mnistm'
    
    labels_fn = os.path.join(path, 'train.csv')
    labels = []
    with open(labels_fn, 'r') as f:
        for index, line in enumerate(f.readlines()):
            if index != 0:
                labels.append(int(line[-2]))
    
    train_fn = os.path.join(path, 'train/*')
    train_files = glob.glob(train_fn)
    train_files.sort()
    train_set = mnistm_dataset(train_files, labels, train_transforms)
    
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=n_workers, pin_memory=True)
    
    return train_loader

# Define Model

In [2]:
def init_params(m):
    class_name = m.__class__.__name__
    if class_name.find('Conv') != -1:
        m.weight.data.normal_(0, 0.02)
        m.bias.data.zero_()

class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.linear = nn.Sequential(
            nn.Linear(110, 256),
            nn.ReLU()
        )

        def convt_bn_relu(in_channels, out_channels, kernel_size, stride):
            return nn.Sequential(
                nn.ConvTranspose2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride),
                nn.BatchNorm2d(out_channels),
                nn.ReLU()
            )
        self.conv_blocks = nn.Sequential(
            convt_bn_relu(256, 192, kernel_size=5, stride=2),
            convt_bn_relu(192, 96, kernel_size=5, stride=2),
        )

        for conv_block in self.conv_blocks:
            for m in conv_block:
                init_params(m)

        self.last_conv = nn.Sequential(
            nn.ConvTranspose2d(96, 3, kernel_size=4, stride=2),
            nn.Tanh()
        )

        for m in self.last_conv:
            init_params(m)


    def forward(self, x):
        x = self.linear(x)
        x = x.view(1, x.shape[0], 1, 1)
        x = self.conv_blocks(x)
        x = self.last_conv(x)
        return x

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.first_conv = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2),
            nn.LeakyReLU(0.2),
            nn.Dropout()
        )
        
        for m in self.first_conv:
            init_params(m)

        def conv_block(in_channels, out_channels, kernel_size, stride):
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride),
                nn.BatchNorm2d(out_channels),
                nn.LeakyReLU(0.2),
                nn.Dropout()
            )
        self.conv_blocks = nn.Sequential(
            conv_block(16, 32, kernel_size=3, stride=1),
            conv_block(32, 64, kernel_size=3, stride=2),
        )

        for conv_block in self.conv_blocks:
            for m in conv_block:
                init_params(m)
        
        self.linear_aux = nn.Linear(64 * 5 * 5, 10)

        self.linear_dis = nn.Sequential(
            nn.Linear(64 * 5 * 5, 1),
            nn.Sigmoid()
        )


    def forward(self, x):
        x = self.first_conv(x)
        x = self.conv_blocks(x)
        x = x.view(x.shape[0], -1)
        
        x_dis = self.linear_dis(x)
        x_aux = self.linear_aux(x)
        return x_dis, x_aux

In [3]:
G = Generator()
D = Discriminator()
print(G)
print(D)

Generator(
  (linear): Sequential(
    (0): Linear(in_features=110, out_features=256, bias=True)
    (1): ReLU()
  )
  (conv_blocks): Sequential(
    (0): Sequential(
      (0): ConvTranspose2d(256, 192, kernel_size=(5, 5), stride=(2, 2))
      (1): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): ConvTranspose2d(192, 96, kernel_size=(5, 5), stride=(2, 2))
      (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
  )
  (last_conv): Sequential(
    (0): ConvTranspose2d(96, 3, kernel_size=(4, 4), stride=(2, 2))
    (1): Tanh()
  )
)
Discriminator(
  (first_conv): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Dropout(p=0.5, inplace=False)
  )
  (conv_blocks): Sequential(
    (0): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
      (1): BatchNorm2d(3

# Training

In [ ]:
def train(device, train_loader, start_epoch, n_epochs, n_critic, z_dim, num_classes, G, optim_G, D, optim_D, z_samples, dis_loss_fn, aux_loss_fn):
    
    for epoch in range(start_epoch, n_epochs):
        
        G.train()
        D.train()

        # train
        # batch_idx = 0
        # for imgs in tqdm(train_loader):
        for batch_idx, batch in enumerate(train_loader):
            
            
            real_imgs, aux_real_labels = batch
            real_imgs = real_imgs.to(device)
            cur_batch_size = real_imgs.shape[0]

            # ============= Train D =============
             
            z_noise = torch.randn((cur_batch_size, z_dim)).to(device)
            aux_fake_labels = torch.randint(num_classes, (cur_batch_size, )).to(device)
            z_onehot = torch.zeros((cur_batch_size, num_classes)).to(device)
            z_onehot[np.arange(cur_batch_size), aux_fake_labels] = 1
            z = torch.cat([z_noise, z_onehot], dim=1).to(device)
            
            fake_imgs = G(z)
            
            dis_fake_preds, aux_fake_preds = D(fake_imgs.detach())
            dis_real_preds, aux_real_preds = D(real_imgs)
            
            dis_fake_labels = torch.zeros((cur_batch_size, 1)).to(device)
            dis_real_labels = torch.ones((cur_batch_size, 1)).to(device)

            
            loss_D = dis_loss_fn(dis_fake_preds, dis_fake_labels) + aux_loss_fn(aux_fake_preds, aux_fake_labels) + \
                    dis_loss_fn(dis_real_preds, dis_real_labels) + aux_loss_fn(aux_real_preds, aux_real_labels)

            D.zero_grad()
            loss_D.backward()
            optim_D.step()

            # ============= Train G =============
            if batch_idx % n_critic == 0:

                dis_fake_preds, aux_fake_preds = D(fake_imgs)

                loss_G = dis_loss_fn(dis_fake_preds, dis_fake_labels) + aux_loss_fn(aux_fake_preds, aux_fake_labels)

                G.zero_grad()
                loss_G.backward()
                optim_G.step()

            # batch_idx += 1
        with open('./record.txt', 'a') as f:
            f.write(f'[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss_D: {loss_D.item():.5f}, loss_G: {loss_G.item():.5f}\n')
        # print(f'loss_D: {loss_D:.5f}, loss_G: {loss_G:.5f}')
        
        # # evaluate
        G.eval()

        test_imgs = (G(z_samples).data + 1) / 2.0

        if not os.path.exists('./output'):
            os.makedirs('./output', exist_ok=True)

        filename = os.path.join('./output', f'Epoch_{epoch + 1:03d}.png')
        torchvision.utils.save_image(test_imgs, filename, nrow=10)
        # print(f'Save some samples to {filename}.')
        with open('./record.txt', 'a') as f:
            f.write(f'Save some samples to {filename}.\n')

        # save checkpoint
        torch.save({'last_epoch': epoch + 1,
                    'model_G': G.state_dict(),
                    'optim_G': optim_G.state_dict(),
                    'model_D': D.state_dict(),
                    'optim_D': optim_D.state_dict(),
                    # 'scheduler': scheduler.state_dict(),
                    }, f'./model_2.ckpt')
        
        with open('./record.txt', 'a') as f:
            f.write('Saving model\n')

# Testing

In [ ]:
def test(G, z_samples, output_repo='./output_repo'):
    # load checkpoint
    ckpt = torch.load(f'./model_2.ckpt', map_location='cpu')
    G.load_state_dict(ckpt['model'])
    G.eval()
    
    # generate images
    generated_imgs = (G(z_samples).data + 1) / 2.0
    
    filenames = []
    for label in range(10):
        for id in range(100):
            filenames.append(f'{label}_{id:03d}.png')

    for i in range(generated_imgs.shape[0]):
        filename = os.path.join(output_repo, filenames[i])
        torchvision.utils.save_image(generated_imgs[i], filename)
    with open('./record.txt', 'a') as f:
        f.write('Save 1000 generated images.\n')

# Start

In [ ]:
# set seed
seed = 0
fix_random_seeds(seed)

# settings
device = "cuda" if torch.cuda.is_available() else "cpu"
n_workers = 0
res = 28
batch_size = 32
n_epochs = 80
n_critic = 1
z_dim = 100
num_classes = 10

z_noise = torch.randn((1000, z_dim))
z_labels = torch.arange(10).repeat((100, ))
z_onehot = torch.zeros((1000, 10))
z_onehot[np.arange(1000), z_labels] = 1
z_samples = torch.cat([z_noise, z_onehot], dim=1).to(device)

load_model = os.path.exists('./model_2.ckpt')

G = Generator().to(device)
D = Discriminator().to(device)
optim_G = torch.optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
optim_D = torch.optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))

dis_loss_fn = nn.BCELoss()
aux_loss_fn = nn.CrossEntropyLoss()

start_epoch = 0

if load_model:
    ckpt = torch.load('./model_2.ckpt')
    start_epoch = ckpt['last_epoch']
    G.load_state_dict(ckpt['model_G'])
    optim_G.load_state_dict(ckpt['optim_G'])
    D.load_state_dict(ckpt['model_D'])
    optim_D.load_state_dict(ckpt['optim_D'])

with open('./record.txt', 'w') as f:
    f.write('')

# training
# if mode == 'train':
with open('record.txt', 'a') as f:
    f.write('start training\n')
image_size = res
train_loader = get_dataset(batch_size, n_workers)
train(device, train_loader, start_epoch, n_epochs, n_critic, z_dim, num_classes, G, optim_G, D, optim_D, z_samples[:100], dis_loss_fn, aux_loss_fn)
with open('record.txt', 'a') as f:
    f.write('finish training\n')

# testing
# if mode == 'test':
test(G, z_samples)

# Test

In [ ]:
class mnistm_dataset(data.Dataset):
    def __init__(self, inputs_path: list, labels: list, transforms):
        self.inputs_path = inputs_path
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.inputs_path)
    
    def __getitem__(self, index: int):
        
        input_path = self.inputs_path[index]
        input = Image.open(input_path)
        label = self.labels[index]

        return self.transforms(input), label

In [ ]:
def get_dataset(batch_size, n_workers):

    train_transforms = transforms.Compose([
        # transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ])
    
    train_fn = './output_repo/*'
    # train_fn = './drive/MyDrive/DLCV/output_repo_2/*'
    train_files = glob.glob(train_fn)
    train_files.sort()
    labels = [int(train_file.split('/')[-1].split('_')[0]) for train_file in train_files]
    train_set = mnistm_dataset(train_files, labels, train_transforms)
    
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=False)
    
    return train_loader

In [ ]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F


def load_checkpoint(checkpoint_path, model):
    state = torch.load(checkpoint_path, map_location = "cpu")
    model.load_state_dict(state['state_dict'])
    print('model loaded from %s' % checkpoint_path)


class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


    
# load digit classifier
net = Classifier()
path = "./drive/MyDrive/DLCV/Classifier.pth"
load_checkpoint(path, net)

# GPU enable
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print('Device used:', device)
if torch.cuda.is_available():
    net = net.to(device)

print(net)

model loaded from ./drive/MyDrive/DLCV/Classifier.pth
Device used: cpu
Classifier(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)


In [ ]:
test_loader = get_dataset(32, 0)
accs = []
for batch in test_loader:
    imgs, labels = batch
    preds = net(imgs.to(device))
    print(preds.argmax(dim=1))
    accs.append((preds.argmax(dim=1) == labels.to(device)).float().mean())
print(sum(accs) / len(accs))

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
tensor([1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2])
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2])
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2,